<div align="center">
    <h1>Projet Algorithmique - Decision Tree </h1>
    <h2>Ikram IDDOUCH </h2>
    
[![Python](https://img.shields.io/badge/python-blue.svg)](https://shields.io/)
[![VisualStudioCode](https://img.shields.io/badge/VisualStudioCode-green.svg)](https://shields.io/)
</div>


<h1>Part 1 : ID3 Algorithm</h1>

<h4>Building the tree</h4>

In [6]:
from collections import Counter
import math
import pandas as pd


In [10]:
def entropy(data):
    """
    This function calculates the entropy of a given dataset.

    Parameters:
        data: The dataset for which entropy needs to be calculated.

    Returns:
        float: The entropy value of the dataset.

    """
    counts = Counter(data)
    entropy = 0
    total = len(data)
    if counts :
        for count in counts.values():
            probability = count / total
            entropy -= probability * math.log2(probability)
    return entropy  


In [11]:
def gain(data, attribute, target):
    """
    This function calculates the information gain when splitting a dataset on a particular attribute.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        attribute (str): The attribute column on which to split the dataset.
        target (str): The target column for classification.

    Returns:
        float: The information gain achieved by splitting the dataset on the given attribute.
    """
    total = entropy(data[target])
    list_attribute = data[attribute].unique()
    attribute_entropy = 0
    
    for value in list_attribute:
        subdata = data[data[attribute] == value]
        attribute_entropy += (len(subdata[target]) / len(data)) * entropy(subdata[target])
    
    gain = total - attribute_entropy
    return gain

In [12]:
def best_attribute(data, target, non_target):
    """
    This function determines the best attribute to split a dataset on, based on the highest information gain.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        target (str): The target column for classification.
        non_target (list): A list of attribute columns to consider for splitting.

    Returns:
        str: The name of the best attribute to split the dataset on.
    """
    gains = {}
    
    for attribute in non_target:
        gains[attribute] = gain(data, attribute, target)
        
    return max(gains, key=gains.get)

In [13]:
def id3(data, target, non_target):
    """
    This function implements the ID3 (Iterative Dichotomiser 3) algorithm for constructing a decision tree.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        target (str): The target column for classification.
        non_target (list): A list of attribute columns to consider for splitting.

    Returns:
        dict: A dictionary representing the decision tree.
    """
    if len(data) == 0:
        return {'No data'}
    
    if not non_target:
        return {target: data[target].mode()[0]} # get the most frequent value
        
    if len(data[target].unique()) == 1:
        return {target: data[target].iloc[0]}
    
    best = best_attribute(data, target, non_target)
    non_target.remove(best)
    
    node = {'Attribute': best, 'Neighbor': {}}
    
    for value in data[best].unique():
        subdata = data[data[best] == value].drop(columns=best)
        node['Neighbor'][value] = id3(subdata, target, non_target.copy())
    
    return node

In [14]:
def print_tree(tree, target,indent=''):
    if target in tree:
        print(indent + target+':', tree[target])
    else:
        print(indent + 'Attribute:', tree['Attribute'])
        for value, child in tree['Neighbor'].items():
            print(indent + '--', value)
            print_tree(child,target,indent + '   ')

Golf Database

In [15]:
golf = pd.read_csv('données/golf.csv')
target_golf = 'play'
non_target_golf = list(golf.keys())
non_target_golf.remove(target_golf)
tree_golf = id3(golf, target_golf, non_target_golf)
print_tree(tree_golf,target_golf)

Attribute: outlook
-- sunny
   Attribute: humidity
   -- high
      play: no
   -- normal
      play: yes
-- overcast
   play: yes
-- rain
   Attribute: wind
   -- False
      play: yes
   -- True
      play: no


Soybean Database

In [16]:
soybean = pd.read_csv('données/soybean-app.csv')

target_soybean = 'hail'
non_target_soybean = list(soybean.keys())
non_target_soybean.remove(target_soybean)
tree_soybean = id3(soybean, target_soybean, non_target_soybean)
print_tree(tree_soybean,target_soybean)

Attribute: lodging
-- yes
   Attribute: class 		
   -- diaporthe-stem-canker
      Attribute: date
      -- august
         Attribute: crop-hist
         -- same-lst-two-yrs
            hail: yes
         -- same-lst-yr
            hail: no
      -- october
         hail: yes
      -- september
         hail: yes
      -- july
         hail: yes
   -- rhizoctonia-root-rot
      hail: yes
   -- phytophthora-rot
      Attribute: external-decay
      -- absent
         hail: yes
      -- firm-and-dry
         hail: no
   -- brown-spot
      Attribute: plant-stand
      -- normal
         hail: yes
      -- lt-normal
         Attribute: area-damaged
         -- low-areas
            Attribute: temp
            -- gt-norm
               hail: no
            -- norm
               hail: yes
         -- scattered
            hail: no
         -- whole-field
            Attribute: temp
            -- norm
               hail: yes
            -- gt-norm
               hail: no
         -- upper

<h4>Confusion matrix 2 classes</h4>

In [18]:
def predict(row,node,target):
    """
    This function predicts the target value for a given row using a decision tree node.

    Parameters:
        row (Series): A single row of data from the dataset.
        node (dict): A dictionary representing a node in the decision tree.
        target (str): The target column for classification.

    Returns:
        The predicted target value for the given row.
    """
    if target in node :
        return node[target]
    else:
        attribute = node['Attribute']
        child = node['Neighbor'][row[attribute]]
        return predict(row,child,target)

In [19]:
def _matrix(predictions, actual):
    """
    This function computes the confusion matrix based on predicted and actual values.

    Parameters:
        predictions (list): A list of predicted target values.
        actual (list): A list of actual target values.

    Returns:
        list: A 2x2 confusion matrix represented as a list of lists.
    """
    true_positives, false_positives, true_negatives, false_negatives = 0, 0, 0, 0
    
    for pred, act in zip(predictions, actual):
        if pred == 'yes' and act == 'yes': 
            true_positives += 1
        elif pred == 'yes' and act == 'no': 
            false_positives += 1
        elif pred == 'no' and act == 'no': 
            true_negatives += 1
        elif pred == 'no' and act == 'yes': 
            false_negatives += 1
    
    matrix = [[true_positives, false_positives], [false_negatives, true_negatives]]
    return matrix

In [20]:
def confusion_matrix1(data,tree,target):
    """
    This function computes the confusion matrix for a given dataset using a decision tree.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        tree (dict): A dictionary representing the decision tree.
        target (str): The target column for classification.

    Returns:
        list: A 2x2 confusion matrix represented as a list of lists
    """
    predictions = []
    for _,row in data.iterrows():
        predictions.append(predict(row,tree,target))
    matrix = _matrix(data[target],predictions) 
    return matrix   

Golf Database

In [21]:
confusion_matrix1(golf,tree_golf,target_golf)

[[9, 0], [0, 5]]

Soybean Database

In [22]:
confusion_matrix1(soybean,tree_soybean,target_soybean)

[[292, 0], [0, 86]]

<h4>Confusion matrix multiclass</h4>

In [23]:
from sklearn.metrics import confusion_matrix

In [24]:
def confusion_matrix_mult(data,tree,target):
    """
    This function computes the confusion matrix for multiclass classification using a decision tree.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        tree (dict): A dictionary representing the decision tree.
        target (str): The target column for classification.

    Returns:
        array: A confusion matrix represented as an array.
    """
    predictions = []
    for _,row in data.iterrows():
        predictions.append(predict(row,tree,target))
    matrix = confusion_matrix(data[target],predictions) 
    return matrix

In [26]:
target_soybean = 'severity'
non_target_soybean = list(soybean.keys())
non_target_soybean.remove(target_soybean)
tree_soybean = id3(soybean, target_soybean, non_target_soybean)
print(confusion_matrix_mult(soybean,tree_soybean,target_soybean))

[[ 77   0   0   0]
 [  0 132   0   0]
 [  0   3 210   0]
 [  0   0   0  33]]


<h1>Part 2 : C4.5 Algorithm </h1>

In [28]:
def split_info(data,attribute):
    """
    This function calculates the split information of a given attribute in a dataset.

    Parameters:
        data (DataFrame): The dataset containing the attribute column.
        attribute (str): The attribute column for which split information needs to be calculated.

    Returns:
        float: The split information value for the attribute.
    """
    int_info = 0
    list_attribute = data[attribute].unique()
    
    for item in list_attribute:
        value = len(data[data[attribute] == item])
        int_info -= value / len(data)*math.log2(value/len(data))
        
    return int_info

In [29]:
def gain_ratio(data, attribute, target):
    """
    This function calculates the gain ratio for a given attribute in a dataset.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        attribute (str): The attribute column for which gain ratio needs to be calculated.
        target (str): The target column for classification.

    Returns:
        float: The gain ratio value for the attribute
    """
    gains = gain(data,attribute,target)
    int_info = split_info(data,attribute)
    if int_info == 0:
        return 0
    return gains / int_info

In [37]:
def best_attribute_c4(data, target, non_target):
    """
    This function determines the best attribute to split a dataset on, based on the highest information gain.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        target (str): The target column for classification.
        non_target (list): A list of attribute columns to consider for splitting.

    Returns:
        str: The name of the best attribute to split the dataset on.
    """
    gains = {}
    
    for attribute in non_target:
        gains[attribute] = gain_ratio(data, attribute, target)
        
    return max(gains, key=gains.get)

In [38]:
def c4_5(data, target, non_target):
    """
    This function implements the C4.5 algorithm, a successor to ID3, for constructing a decision tree.

    Parameters:
        data (DataFrame): The dataset containing both attribute and target columns.
        target (str): The target column for classification.
        non_target (list): A list of attribute columns to consider for splitting.

    Returns:
        dict: A dictionary representing the decision tree.
    """
    if len(data) == 0:
        return {'No data'}
    
    if not non_target:
        return {target: data[target].mode()[0]}
        
    if len(data[target].unique()) == 1:
        return {target: data[target].iloc[0]}
    
    best = best_attribute_c4(data, target, non_target)
    non_target.remove(best)
    
    node = {'Attribute': best, 'Neighbor': {}}
    
    for value in data[best].unique():
        subdata = data[data[best] == value].drop(columns=best)
        node['Neighbor'][value] = c4_5(subdata, target, non_target.copy())
    
    return node

Golf Database

In [39]:
golf = pd.read_csv('données/golf.csv')
target_golf = 'play'
non_target_golf = list(golf.keys())
non_target_golf.remove(target_golf)
tree_golf = c4_5(golf, target_golf, non_target_golf)
print_tree(tree_golf,target_golf)

Attribute: outlook
-- sunny
   Attribute: humidity
   -- high
      play: no
   -- normal
      play: yes
-- overcast
   play: yes
-- rain
   Attribute: wind
   -- False
      play: yes
   -- True
      play: no


Soybean Database

In [40]:
df = pd.read_csv('données/soybean-app.csv')
df.replace('?', pd.NA, inplace=True)

soybean = df.dropna()
soybean.reset_index(drop=True, inplace=True)

target_soybean = 'severity'
non_target_soybean = list(soybean.keys())
non_target_soybean.remove(target_soybean)
tree_soybean = c4_5(soybean, target_soybean, non_target_soybean)
print_tree(tree_soybean,target_soybean)

Attribute: mycelium
-- absent
   Attribute: plant-growth
   -- abnorm
      Attribute: leaf-malf
      -- absent
         Attribute: shriveling
         -- absent
            Attribute: fruit-spots
            -- dna
               Attribute: sclerotia
               -- absent
                  Attribute: date
                  -- august
                     Attribute: plant-stand
                     -- normal
                        Attribute: hail
                        -- yes
                           severity: severe
                        -- no
                           Attribute: crop-hist
                           -- same-lst-yr
                              severity: pot-severe
                           -- same-lst-two-yrs
                              severity: pot-severe
                           -- same-lst-sev-yrs
                              severity: severe
                     -- lt-normal
                        severity: pot-severe
                  -- october